
![Biofilm image](../images/Biofilm_Website_2.png)

# Submodule #2: Microbiome Analysis

## Overview
One of the primary objectives of the workflow is to characterize the taxonomic diversity of biofilm communities from 16S data using Qiime2. This step helps provide insights into the microbial diversity by identifying and associating specific organisms or taxonomic groups with phenotypic/functional traits characterizing a given environment. Taxonomic classification is challenging because the volume of metagenomics data is large and puts high demands on computational resources. Additionally, queried sequences of most microbes lack taxonomically related sequences in existing references databases. **Taxonomic binning**, the process of assigning taxonomic identifiers to sequence fragments based on sequence similarity and composition, is used to reconstruct draft genomes. The outcome of the binning process can then be used not only for taxonomic diversity assessment, but also for genome assembly and evaluation of gene association across different taxonomies. 

# Learning Objectives:
During Submodule 2 we will use our dataset from [**Submodule 1**](./SubModule01.ipynb) to perform microbiome analysis using the tool Qiime2. At the end of this submodule, the learner will be able to:
- Analyze sequencing datasets and map reads to relevant microbiome taxonomy
- Retrieve and understand the microbiome community available on the input dataset (alpha and beta diversity)
- Interpret microbiome analysis results from canonical tools such as Qiime2

## Prerequisites

*   **Software:**
    *   [Qiime2](https://qiime2.org/)
    *   Mamba
*   **Data:**
    *   `Core_Dataset_Prep/sample-metadata.tsv`:  Sample metadata in a TSV format.
    *   `Core_Dataset_Prep/emp-single-end-sequences`: Directory containing sequence and barcode FASTQ files.
    *   `qiime2_analysis/scripts/qiime-classifier/gg-13-8-99-515-806-nb-classifier.qza`: Pre-trained Naive Bayes classifier for taxonomic assignment.




<div class="alert alert-block alert-info">
    <i class="fa fa-lightbulb-o" aria-hidden="true"></i>
    <b>Note: </b> The scope of this training tutorial focuses on the 16S metagenomics with a single SE demultiplexed dataset, but the user can customize the script to analyze other metagenomics data modalities such as Shotgun (e.g. test <a href="./samples_shotgun_analysis.ipynb">HERE</a>) leveraging QIIME documentation and forum.
</div>

## Get Started
### Step 3 - Microbiome Analysis with Qiime2 (duration ~ 20 mins): 

### Install Qiime2

Run the following cell to install Qiime2.

In [ ]:
! mamba create -n qiime2 -c https://packages.qiime2.org/qiime2/2022.11/passed/core/ -c conda-forge -c bioconda qiime2-core -y

In [ ]:
# Add qiime2 to the PATH
import os
os.environ['PATH'] = '/home/ec2-user/anaconda3/envs/qiime2/bin:' + os.environ['PATH']

### Run the Qiime2 Example Pipeline - Moving Pictures

The following script is from the Qiime2 tutorial workshop that looks into two human samples at four different body sites. Typically when running a metagenomic analysis you may not know what is in your sample. To help identify them, sequences are tagged with barcodes that can be looked up in a corresponding metadata table alongside other relevant information (e.g. location collected, data, time, sample it came from etc.). In the end, you will have three sets of inputs:
- **Sequence file** Contains all the sequences found in your sample.
- **Barcode file** Contains all the barcodes each sequence is tagged with.
- **Metadata file** Contains sample information and each sample's barcode. Ours holds the following information:
    - The body site where the sample was collected
    - The sample ID
    - The time and date the sample was collected

### Analysis Steps
1. `demux` demultiplexes the sequences. This means it will identify which barcode is associated with each sample which will in turn help us identify which sequences are assocaited with each sample. 

2. `DADA2` identifies and filters out any phiX and chimeric sequences in other words it will trim any low quality reads.   The end result will produce:
    - A FeatureTable that holds counts of each unique sequence in each sample within our dataset. 
    - A FeatureData that maps feature identifiers in the FeatureTable to the sequences they represent. 
<br>
<br>
3. `core-metrics-phylogenetic` computes alpha and beta diversity metrics.

4. `alpha-group-significance` tests for associations between the metadata columns that are labeled as categorical and alpha diversity data. The same will be done with the beta diversity data via the `beta-group-significance`.

5. `emperor` will create principal coordinates (PCoA) plots from distance matrices that were created during beta diversity analysis. These plots visualize relationships between samples based on the similarity of their microbiome profile.

6. `feature-classifier` will predict the taxonomic composition of the samples in the FeatureData file based on a pre-trained Naive Bayes classifier that we have provided.

7. `feature-table filter-samples` conducts a differential abundance analysis using ANCOM that specifically looks at the features that are differentially abundant across sample groups that are associated with the gut. This step also holds many sub processes:
    - Creates a feature table that contains only the gut samples
    - Creates a Features Counts table
    - We then run ANCOM to identify the difference of abundance based on the features from our metadata between the two subjects.

This script took up to 10 minutes to run on our test data. It also contains commands to produce visuals for almost each step that can be viewed here in your Jupyter notebook. To learn more about this script visit: https://github.com/qiime2/docs/blob/master/source/tutorials/moving-pictures.rst

### Prepare Data for Qiime2 Analysis

The following script has been modified to allow you to enter in your own data as desired, the following options for this script are:

**-m**: Metadata file <br>
**-s**: Directory that contains sequence and barcode fastq files <br>
**-b**: Body site <br>
**-c**: Classifier file <br>
**-o**: Output directory <br>
**-l*: Trims the left side of the fastq file <br>
**-t**: Trims from the specified position and onward

Remember that we need to truncate from bp 120 and onward just as we saw in Submodule 2 thats where our bp quality goes down. 

In [ ]:
! bash qiime2_analysis/scripts/moving-picture.sh \
    -m Core_Dataset_Prep/sample-metadata.tsv \
    -s Core_Dataset_Prep/emp-single-end-sequences \
    -b "gut" \
    -c qiime2_analysis/scripts/qiime-classifier/gg-13-8-99-515-806-nb-classifier.qza \
    -o qiime2_analysis/qiime2_Output \
    -l "0" \
    -t "120"

### Post Process (Uncompress zip files, Create Folder and Save Output)

Following Qiime2 analysis, the results are in compressed files that need to be extracted before visualization or loaded into a downstream tools such as: MicrobiomeAnalyst, Qiime Viewer, R, etc. The following script will unzip all outputs in our qiime2_Output directory.

In [ ]:
! bash qiime2_analysis/scripts/qiime-postprocessing.sh -o qiime2_analysis/qiime2_Output

<div class="alert alert-block alert-danger">
    <i class="fa fa-exclamation-circle" aria-hidden="true"></i>
    <b>Alert: </b> Unfortunately Qiime2 does not let you overwrite output files. If you would like to rerun this analysis again make sure you delete the contents within the output folder via the command:
    
    rm -r qiime2_analysis/qiime2_Output
    
The script will make your output directory automatically.
</div> 

At the completion of our Qiime analysis and post-processing, all output are uncompressed in Qiime's custom output format. This step allow us to extract the content and have access to the readable format such as HTML, csv, and figures in our qiime2_Output directory. **You will notice that any directory that holds the notation **'viz'** contains visuals such as plots and figures that you can explore!**

### Exploring Results

<div class="alert alert-block alert-info">
    <i class="fa fa-lightbulb-o" aria-hidden="true"></i>
    <b>NOTE: </b> To view HTML result files, right-click a <i>.qzv</i> file within the <i>qiime2_analysis/qiime2_Output/core-metrics-results</i> directory and download it. Next, rename the downloaded file, changing the extension from <i>.qzv</i> to </i>zip</i>. Unzip this file and then open HTML file located inside the <i>data</i> directory.
</div>

We mentioned **alpha diversity** and **beta diversity** before but what are they? Alpha diversity is the measure of microbiome diversity within a single sample and beta diversity is the measure of similarities or dissimilarities between two communities.

Lets view some of our results starting with analyzing the alpha diversity distribution in `qiime2_analysis/qiime2_Output/core-metrics-results/faith-pd-group-significance.qzv`.

![faith](../images/faith-pd-group-significance.png)

Next we will view our beta diversity distribution. The HTML is fully interactive to allow you to see the different dimensions of the PCoA plot just click, hold, and move your curser around to view the different clusters.

![emperor](../images/bray_curtis_emperor.png)

### Quiz

In [ ]:
#Run the command below to take the quiz!
from IPython.display import IFrame
IFrame("../Quiz/QS13.html", width=800, height=350)

## Conclusion

We have characterized the taxonomic diversity of biofilm communities from 16s dataset using Qiime2. This step provided insights into the microbial diversity. We identified the specific organisms or taxonomic groups and associated them with phenotypic/functional traits characterized by a given environment. The taxonomic binning process was used for taxonomic diversity analysis and was used for genome assembly followed by evaluation of gene associations with different taxonomies.

## Clean up

Remember to stop your notebook instance when you are done!